In [17]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint



def estBetaParams(mu, var):
    alpha=((1 - mu) / var - 1 / mu) * mu**2
    beta=alpha * (1 / mu - 1)
    return alpha,beta

def estGammaParams(mu,var):
    shape=(mu**2)/var
    scale=var/mu
    return shape, scale

    #Non-Stochastic Parameters Parameters

N = 10**6           #250000               # Total population
p_3months = 0.3          # Proportion tested every 3 months
p_12months = 1-p_3months         # Proportion tested every 12 months
tau = 7                # This is our key parameter; the average return time for tests.

# The (weighted) average testing proportion for our population (proportion per day)
average_daily_testing_proportion = (p_3months/91 + p_12months/365)
tests_per_day=N*average_daily_testing_proportion
# I think that the generic person will get tested with this probability each day, and so the waiting
# time for a test from any given instant is 1/average_testing_proportion (exponential waiting time)
# No.... maybe it should be with rate tests_per_day??

# Initial conditions
E0 = 0.1 * N             # Initial exposed individuals.
X0 = N - E0              # Initial susceptible individuals.
S0 = 0                   # Initial symptomatic individuals (we assume the symptoms are NOTICEABLE). Symptomatic Individuals are assumed to isolate.
A_u0 = 0               # Initial asymptomatic individuals who are untested.
A_t0 = 0                 # Initial asympt. who are tested and awating a positive. These individuals will NOT isolate. 
A_pos0 = 0                 # Initial asympt. positive test. These people are isolating. 
                   # Initial people with complications 


#Define the Model 

def model(Y, t, beta, epsilon, gamma_t, gamma_u, lambda_, theta, omega_r, average_daily_testing_proportion , tau, N):


    X, E, S, A_u, A_t, A_pos= Y


    # Here, we assume that the only groups driving the spread of infection are the asymptomatic who have not received a positive test
    dXdt = -beta*X*(A_u+A_t)/N + gamma_t*S  + gamma_t*A_pos + gamma_u*A_t + (1-theta)*gamma_u*A_u


    dEdt = beta*X*(A_u+A_t)/N - epsilon*E


    dSdt = (1-lambda_)*epsilon*E + theta*gamma_u*A_u - gamma_t*S 


    dA_udt = lambda_*epsilon*E - average_daily_testing_proportion*A_u - gamma_u*A_u #I want some proportion to clear, and some proportion to become infectious

    dA_tdt = average_daily_testing_proportion*A_u - omega_r*A_t - gamma_u*A_t


    dA_posdt = omega_r*A_t - gamma_t*A_pos




    return [dXdt, dEdt, dSdt, dA_udt, dA_tdt, dA_posdt]





In [30]:
tau=14
num_trials=500



t_points = np.linspace(0, 50 * 365, 50 * 365) #25 years


# ODE solver parameters
abserr = 1.0e-8
relerr = 1.0e-6


solutions=np.zeros((num_trials,len(t_points), 6))
sim_params=np.zeros((num_trials, len(t_points),8))

for i in range(num_trials):
    solutions[i][0]= [X0, E0, S0, A_u0, A_t0, A_pos0]

for i in range(num_trials):
    for j in range(1, len(t_points)):

        #Stochastic params
        beta = np.random.beta(estBetaParams(0.9,0.01)[0],estBetaParams(0.9,0.01)[1])*np.random.gamma(estGammaParams(2.85,1)[0],estGammaParams(2.85,1)[1])/30  
        epsilon = 1/(np.random.gamma(estGammaParams(5,1)[0],estGammaParams(5,1)[1])+0.000000000000001) 
        gamma_t = 1/(np.random.gamma(estGammaParams(7,1)[0],estGammaParams(7,1)[1])+0.00000000000001) 
        theta=np.random.beta(estBetaParams(0.1,0.01)[0],estBetaParams(0.1,0.01)[1])
        gamma_u = 1/(np.random.gamma(estGammaParams(14,3)[0],estGammaParams(14,3)[1])+0.00000000000001) 
        lambda_ = np.random.beta(estBetaParams(0.9,0.01)[0],estBetaParams(0.9,0.01)[1]) 
        omega_r = 1/(np.random.gamma(estGammaParams(tau,1)[0], estGammaParams(tau,1)[1])+0.00000000000001)
        eta = np.random.beta(estBetaParams(0.0225,0.0075**2)[0],estBetaParams(0.0225,0.0075**2)[1])

        #record these for average yearly rates later.
        sim_params[i][j]=[beta,epsilon,gamma_t,theta,gamma_u,lambda_,omega_r,eta]


        tspan = [t_points[j-1], t_points[j]]

        ys = odeint(model, solutions[i][j-1], tspan, args=(beta, epsilon, gamma_t, gamma_u, lambda_, theta, omega_r, average_daily_testing_proportion, tau, N), atol=abserr, rtol=relerr)

         # Update the solution
        solutions[i][j] = ys[-1]
        #print(i,j, solutions[i][j], sum(solutions[i][j]))

    #Extract_Last_year_data

    last_year_data=[solution[-365:] for solution in solutions]
    last_year_params=[params[-365:] for params in sim_params]

    #rescale by the population size to get a relativized version:

    last_year_data = [[value / N for value in day] for day in last_year_data]

    # Average Solution: we shoudld compute the totals for each year before averaging in order to get better stats

    E_totals=[]
    S_totals=[]
    A_u_totals=[]
    A_t_totals=[]
    A_pos_totals=[]
    Infection_totals=[]
    Detection_totals=[]
    Percent_detection_totals=[]

    for i in range(num_trials): 

            # Initialize variables to store the sum of daily inflows
        total_yearly_inflow_E = 0
        total_yearly_inflow_S = 0
        total_yearly_inflow_A_u = 0
        total_yearly_inflow_A_t = 0
        total_yearly_inflow_A_pos = 0

        # Calculate daily inflow rates for the last year
        for j in range(365):
            X, E, S, A_u, A_t, A_pos = last_year_data[i][j]
            beta, epsilon, gamma_t, theta, gamma_u, lambda_, omega_r, eta = last_year_params[i][j]

            # Compute daily inflow rates
            daily_inflow_E = beta * X * (A_u + A_t) / N
            daily_inflow_S = (1 - lambda_) * epsilon * E + theta * gamma_u * A_u
            daily_inflow_A_u = lambda_ * epsilon * E
            daily_inflow_A_t = average_daily_testing_proportion * A_u
            daily_inflow_A_pos = omega_r * A_t

            # Sum the inflows
            total_yearly_inflow_E += daily_inflow_E
            total_yearly_inflow_S += daily_inflow_S
            total_yearly_inflow_A_u += daily_inflow_A_u
            total_yearly_inflow_A_t += daily_inflow_A_t
            total_yearly_inflow_A_pos += daily_inflow_A_pos


        # Total infections for the last year
        total_infections_last_year = total_yearly_inflow_E 
        # Total Detected Cases 
        total_detected_cases = total_yearly_inflow_S + total_yearly_inflow_A_pos

        # Percentage of Cases Detected: 
        percent_detected_cases=total_detected_cases/total_infections_last_year

        E_totals.append(total_yearly_inflow_E)
        S_totals.append(total_yearly_inflow_S)
        A_u_totals.append(total_yearly_inflow_A_u)
        A_t_totals.append(total_yearly_inflow_A_t)
        A_pos_totals.append(total_yearly_inflow_A_pos)
        Infection_totals.append(total_infections_last_year)
        Detection_totals.append(total_detected_cases)
        Percent_detection_totals.append(percent_detected_cases)



#now compute averages accross all simulations:
from scipy import stats


mean_E_total=np.mean(E_totals)
stDev_E_total= np.std(E_totals)
conf_E_total = stats.norm.interval(0.95, loc=mean_E_total, scale=stDev_E_total)
E_total_stats=[mean_E_total, stDev_E_total,conf_E_total]

mean_S_total=np.mean(S_totals)
stDev_S_total= np.std(S_totals)
conf_S_total = stats.norm.interval(0.95, loc=mean_S_total, scale=stDev_S_total)
S_total_stats=[mean_S_total, stDev_S_total,conf_S_total]

mean_A_u_total=np.mean(A_u_totals)
stDev_A_u_total= np.std(A_u_totals)
conf_A_u_total = stats.norm.interval(0.95, loc=mean_A_u_total, scale=stDev_A_u_total)
A_u_total_stats=[mean_A_u_total, stDev_A_u_total,conf_A_u_total]

mean_A_t_total=np.mean(A_t_totals)
stDev_A_t_total= np.std(A_t_totals)
conf_A_t_total = stats.norm.interval(0.95, loc=mean_A_t_total, scale=stDev_A_t_total)
A_t_total_stats=[mean_A_t_total, stDev_A_t_total,conf_A_t_total]

mean_A_pos_total=np.mean(A_pos_totals)
stDev_A_pos_total= np.std(A_pos_totals)
conf_A_pos_total = stats.norm.interval(0.95, loc=mean_A_pos_total, scale=stDev_A_pos_total)
A_pos_total_stats=[mean_A_pos_total, stDev_A_pos_total,conf_A_pos_total]

mean_Infection_total=np.mean(Infection_totals)
stDev_Infection_total= np.std(Infection_totals)
conf_Infection_total = stats.norm.interval(0.95, loc=mean_Infection_total, scale=stDev_Infection_total)
Infection_total_stats=[mean_Infection_total, stDev_Infection_total,conf_Infection_total]

mean_Detection_total=np.mean(Detection_totals)
stDev_Detection_total= np.std(Detection_totals)
conf_Detection_total = stats.norm.interval(0.95, loc=mean_Detection_total, scale=stDev_Detection_total)
Detection_total_stats=[mean_Detection_total, stDev_Detection_total,conf_Detection_total]


mean_Percent_detection_total=np.mean(Percent_detection_totals)
stDev_Percent_detection_total= np.std(Percent_detection_totals)
conf_Percent_detection_total = stats.norm.interval(0.95, loc=mean_Percent_detection_total, scale=stDev_Percent_detection_total)
Percent_detection_total_stats=[mean_Percent_detection_total, stDev_Percent_detection_total,conf_Percent_detection_total]


<ipython-input-30-700df8eea733>:100: RuntimeWarning: invalid value encountered in scalar divide
  percent_detected_cases=total_detected_cases/total_infections_last_year


In [31]:
Detection_total_stats

[0.09100907970711597,
 0.040546143294073106,
 (0.011540099138732454, 0.1704780602754995)]

In [32]:
Infection_total_stats

[4.2715701361048887e-07,
 1.9142394536455696e-07,
 (5.197297491739417e-08, 8.023410523035836e-07)]

In [33]:
np.std(Detection_totals)

0.040546143294073106

In [19]:
0.04198498947665725*250000 #tau=7

10496.247369164312

In [23]:
0.003504716690125117* 250000 #tau = 3

876.1791725312793

In [27]:
8.623262340171376e-06*250000 #tau=1

2.155815585042844

In [34]:
0.09100907970711597*250000 #tau = 14

22752.269926778994

In [36]:
detections=[]

for tau in range(1,10):
    
    num_trials=500


    t_points = np.linspace(0, 50 * 365, 50 * 365) #25 years


    # ODE solver parameters
    abserr = 1.0e-8
    relerr = 1.0e-6


    solutions=np.zeros((num_trials,len(t_points), 6))
    sim_params=np.zeros((num_trials, len(t_points),8))

    for i in range(num_trials):
        solutions[i][0]= [X0, E0, S0, A_u0, A_t0, A_pos0]

    for i in range(num_trials):
        for j in range(1, len(t_points)):

            #Stochastic params
            beta = np.random.beta(estBetaParams(0.9,0.01)[0],estBetaParams(0.9,0.01)[1])*np.random.gamma(estGammaParams(2.85,1)[0],estGammaParams(2.85,1)[1])/30  
            epsilon = 1/(np.random.gamma(estGammaParams(5,1)[0],estGammaParams(5,1)[1])+0.000000000000001) 
            gamma_t = 1/(np.random.gamma(estGammaParams(7,1)[0],estGammaParams(7,1)[1])+0.00000000000001) 
            theta=np.random.beta(estBetaParams(0.1,0.01)[0],estBetaParams(0.1,0.01)[1])
            gamma_u = 1/(np.random.gamma(estGammaParams(14,3)[0],estGammaParams(14,3)[1])+0.00000000000001) 
            lambda_ = np.random.beta(estBetaParams(0.9,0.01)[0],estBetaParams(0.9,0.01)[1]) 
            omega_r = 1/(np.random.gamma(estGammaParams(tau,1)[0], estGammaParams(tau,1)[1])+0.00000000000001)
            eta = np.random.beta(estBetaParams(0.0225,0.0075**2)[0],estBetaParams(0.0225,0.0075**2)[1])

            #record these for average yearly rates later.
            sim_params[i][j]=[beta,epsilon,gamma_t,theta,gamma_u,lambda_,omega_r,eta]


            tspan = [t_points[j-1], t_points[j]]

            ys = odeint(model, solutions[i][j-1], tspan, args=(beta, epsilon, gamma_t, gamma_u, lambda_, theta, omega_r, average_daily_testing_proportion, tau, N), atol=abserr, rtol=relerr)

             # Update the solution
            solutions[i][j] = ys[-1]
            #print(i,j, solutions[i][j], sum(solutions[i][j]))

        #Extract_Last_year_data

        last_year_data=[solution[-365:] for solution in solutions]
        last_year_params=[params[-365:] for params in sim_params]

        #rescale by the population size to get a relativized version:

        last_year_data = [[value / N for value in day] for day in last_year_data]

        # Average Solution: we shoudld compute the totals for each year before averaging in order to get better stats

        E_totals=[]
        S_totals=[]
        A_u_totals=[]
        A_t_totals=[]
        A_pos_totals=[]
        Infection_totals=[]
        Detection_totals=[]
        Percent_detection_totals=[]

        for i in range(num_trials): 

                # Initialize variables to store the sum of daily inflows
            total_yearly_inflow_E = 0
            total_yearly_inflow_S = 0
            total_yearly_inflow_A_u = 0
            total_yearly_inflow_A_t = 0
            total_yearly_inflow_A_pos = 0

            # Calculate daily inflow rates for the last year
            for j in range(365):
                X, E, S, A_u, A_t, A_pos = last_year_data[i][j]
                beta, epsilon, gamma_t, theta, gamma_u, lambda_, omega_r, eta = last_year_params[i][j]

                # Compute daily inflow rates
                daily_inflow_E = beta * X * (A_u + A_t) / N
                daily_inflow_S = (1 - lambda_) * epsilon * E + theta * gamma_u * A_u
                daily_inflow_A_u = lambda_ * epsilon * E
                daily_inflow_A_t = average_daily_testing_proportion * A_u
                daily_inflow_A_pos = omega_r * A_t

                # Sum the inflows
                total_yearly_inflow_E += daily_inflow_E
                total_yearly_inflow_S += daily_inflow_S
                total_yearly_inflow_A_u += daily_inflow_A_u
                total_yearly_inflow_A_t += daily_inflow_A_t
                total_yearly_inflow_A_pos += daily_inflow_A_pos


            # Total infections for the last year
            total_infections_last_year = total_yearly_inflow_E 
            # Total Detected Cases 
            total_detected_cases = total_yearly_inflow_S + total_yearly_inflow_A_pos

            # Percentage of Cases Detected: 
            percent_detected_cases=total_detected_cases/total_infections_last_year

            E_totals.append(total_yearly_inflow_E)
            S_totals.append(total_yearly_inflow_S)
            A_u_totals.append(total_yearly_inflow_A_u)
            A_t_totals.append(total_yearly_inflow_A_t)
            A_pos_totals.append(total_yearly_inflow_A_pos)
            Infection_totals.append(total_infections_last_year)
            Detection_totals.append(total_detected_cases)
            Percent_detection_totals.append(percent_detected_cases)



    #now compute averages accross all simulations:
    from scipy import stats


    mean_E_total=np.mean(E_totals)
    stDev_E_total= np.std(E_totals)
    conf_E_total = stats.norm.interval(0.95, loc=mean_E_total, scale=stDev_E_total)
    E_total_stats=[mean_E_total, stDev_E_total,conf_E_total]

    mean_S_total=np.mean(S_totals)
    stDev_S_total= np.std(S_totals)
    conf_S_total = stats.norm.interval(0.95, loc=mean_S_total, scale=stDev_S_total)
    S_total_stats=[mean_S_total, stDev_S_total,conf_S_total]

    mean_A_u_total=np.mean(A_u_totals)
    stDev_A_u_total= np.std(A_u_totals)
    conf_A_u_total = stats.norm.interval(0.95, loc=mean_A_u_total, scale=stDev_A_u_total)
    A_u_total_stats=[mean_A_u_total, stDev_A_u_total,conf_A_u_total]

    mean_A_t_total=np.mean(A_t_totals)
    stDev_A_t_total= np.std(A_t_totals)
    conf_A_t_total = stats.norm.interval(0.95, loc=mean_A_t_total, scale=stDev_A_t_total)
    A_t_total_stats=[mean_A_t_total, stDev_A_t_total,conf_A_t_total]

    mean_A_pos_total=np.mean(A_pos_totals)
    stDev_A_pos_total= np.std(A_pos_totals)
    conf_A_pos_total = stats.norm.interval(0.95, loc=mean_A_pos_total, scale=stDev_A_pos_total)
    A_pos_total_stats=[mean_A_pos_total, stDev_A_pos_total,conf_A_pos_total]

    mean_Infection_total=np.mean(Infection_totals)
    stDev_Infection_total= np.std(Infection_totals)
    conf_Infection_total = stats.norm.interval(0.95, loc=mean_Infection_total, scale=stDev_Infection_total)
    Infection_total_stats=[mean_Infection_total, stDev_Infection_total,conf_Infection_total]

    mean_Detection_total=np.mean(Detection_totals)
    stDev_Detection_total= np.std(Detection_totals)
    conf_Detection_total = stats.norm.interval(0.95, loc=mean_Detection_total, scale=stDev_Detection_total)
    Detection_total_stats=[mean_Detection_total, stDev_Detection_total,conf_Detection_total]


    mean_Percent_detection_total=np.mean(Percent_detection_totals)
    stDev_Percent_detection_total= np.std(Percent_detection_totals)
    conf_Percent_detection_total = stats.norm.interval(0.95, loc=mean_Percent_detection_total, scale=stDev_Percent_detection_total)
    Percent_detection_total_stats=[mean_Percent_detection_total, stDev_Percent_detection_total,conf_Percent_detection_total]
    
    detections.append(mean_detection_total*250000)


<ipython-input-36-8689b7a9c41a>:102: RuntimeWarning: invalid value encountered in scalar divide
  percent_detected_cases=total_detected_cases/total_infections_last_year


NameError: name 'mean_detection_total' is not defined

In [ ]:
plt.plot(detections)